In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, BatchNormalization, Concatenate, Flatten
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

c:\Users\adupa\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\adupa\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\adupa\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\adupa\AppD

In [ ]:
data_path = 'Final Fashion Dataset.csv'  
df = pd.read_csv(data_path)

In [3]:
df

,user_id,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,filename,link,ratings,review,Month,Price (USD)
0,93810,8493,Women,Apparel,Bottomwear,Skirts,Black,Fall,2011.0,Casual,Forever New Women Black Skirts,8493.jpg,http://assets.myntassets.com/v1/images/style/p...,5,Amazing quality! Definitely recommend. Would r...,November,46
1,24592,30757,Men,Apparel,Topwear,Kurtas,Grey,Summer,2012.0,Ethnic,Fabindia Men Grey Mangalgiri Kurta,30757.jpg,http://assets.myntassets.com/v1/images/style/p...,4,"Perfect for any occasion, great buy..",September,29
2,13278,14881,Women,Accessories,Bags,Handbags,Black,Summer,2011.0,Casual,United Colors of Benetton Women Solid Black Ha...,14881.jpg,http://assets.myntassets.com/v1/images/style/p...,3,"It's okay, nothing special.!",October,39
3,46048,48449,Men,Apparel,Topwear,Tshirts,Blue,Summer,2012.0,Casual,French Connection Men Blue T-shirt,48449.jpg,http://assets.myntassets.com/v1/images/style/p...,5,"Comfortable and stylish, worth the price. and ...",August,39
4,42098,4697,Unisex,Accessories,Watches,Watches,Black,Winter,2016.0,Sports,ADIDAS Unisex Digital Duramo Black Watch,4697.jpg,http://assets.myntassets.com/v1/images/style/p...,4,"Perfect for any occasion, great buy.!",March,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36479,40311,45378,Women,Footwear,Shoes,Flats,Black,Summer,2012.0,Casual,Rocia Women Black Sandals,45378.jpg,http://assets.myntassets.com/v1/images/style/p...,5,"High quality fabric, very satisfied. and I'm v...",November,48
36480,93769,8219,Unisex,Accessories,Bags,Backpacks,Red,Winter,2016.0,Casual,Fastrack Unisex Navy Red Single Strap Backpack,8219.jpg,http://assets.myntassets.com/v1/images/style/p...,4,"Exceeded expectations, love the fit! but could...",February,26
36481,77537,35177,Unisex,Accessories,Bags,Backpacks,Black,Summer,2015.0,Casual,Wildcraft Unisex Black Backpack,35177.jpg,http://assets.myntassets.com/v1/images/style/p...,1,"Poor material, doesn't last long. but could be...",August,40
36482,80371,8673,Men,Accessories,Eyewear,Sunglasses,Black,Winter,2016.0,Casual,Fastrack Men metal rim Black Sunglasses,8673.jpg,http://assets.myntassets.com/v1/images/style/p...,1,"Poor material, doesn't last long. Would not bu...",March,30


In [4]:
complementary_map = {
    'Topwear': ['Bottomwear', 'Shoes', 'Flip Flops', 'Jewellery', 'Eyewear', 'Belts', 'Bags', 'Watches', 'Wallets'],
    'Bottomwear': ['Topwear', 'Shoes', 'Flip Flops', 'Bags', 'Belts', 'Watches', 'Wallets'],
    'Bags': ['Topwear', 'Bottomwear', 'Shoes', 'Watches', 'Jewellery', 'Eyewear'],
    'Watches': ['Topwear', 'Bottomwear', 'Bags', 'Shoes', 'Jewellery'],
    'Shoes': ['Topwear', 'Bottomwear', 'Flip Flops', 'Socks', 'Eyewear', 'Belts'],
    'Flip Flops': ['Topwear', 'Bottomwear', 'Socks', 'Belts', 'Eyewear'],
    'Jewellery': ['Topwear', 'Saree', 'Watches', 'Eyewear'],
    'Eyewear': ['Topwear', 'Bottomwear', 'Shoes', 'Jewellery'],
    'Belts': ['Topwear', 'Bottomwear', 'Shoes', 'Flip Flops', 'Watches'],
    'Saree': ['Jewellery', 'Bags', 'Shoes'],
    'Loungewear and Nightwear': ['Bottomwear', 'Socks'],
    'Wallets': ['Topwear', 'Bottomwear', 'Bags', 'Watches'],
    'Socks': ['Shoes', 'Flip Flops', 'Loungewear and Nightwear']
}

In [5]:
colour_contrast_map = {
    'Black': ['White', 'Grey', 'Beige', 'Silver', 'Cream'],
    'Grey': ['Black', 'White', 'Red', 'Navy Blue'],
    'Blue': ['White', 'Cream', 'Yellow', 'Pink', 'Silver'],
    'Pink': ['Navy Blue', 'White', 'Beige', 'Grey'],
    'Brown': ['Cream', 'White', 'Beige', 'Olive', 'Mustard'],
    'Cream': ['Black', 'Blue', 'Brown', 'Navy Blue', 'Purple'],
    'Green': ['White', 'Black', 'Navy Blue', 'Yellow', 'Brown'],
    'White': ['Black', 'Blue', 'Red', 'Navy Blue', 'Grey'],
    'Navy Blue': ['White', 'Yellow', 'Cream', 'Pink', 'Beige'],
    'Yellow': ['Black', 'Navy Blue', 'Grey', 'Green', 'Purple'],
    'Silver': ['Black', 'Blue', 'Pink', 'Red'],
    'Red': ['White', 'Black', 'Grey', 'Beige'],
    'Beige': ['Navy Blue', 'Black', 'Red', 'Olive', 'Cream'],
    'Maroon': ['Cream', 'Beige', 'White', 'Olive'],
    'Gold': ['Black', 'White', 'Cream', 'Navy Blue'],
    'Magenta': ['White', 'Navy Blue', 'Cream'],
    'Lavender': ['Navy Blue', 'White', 'Grey'],
    'Multi': ['White', 'Black'],
    'Purple': ['Yellow', 'White', 'Cream'],
    'Charcoal': ['White', 'Cream', 'Blue'],
    'Orange': ['White', 'Black', 'Navy Blue'],
    'Tan': ['Navy Blue', 'White', 'Black'],
    'Olive': ['White', 'Yellow', 'Cream', 'Tan'],
    'Off White': ['Black', 'Grey', 'Navy Blue'],
    'Grey Melange': ['Black', 'White', 'Red', 'Blue'],
    'Rust': ['White', 'Cream', 'Beige', 'Navy Blue'],
    'Turquoise Blue': ['White', 'Black', 'Yellow'],
    'Mustard': ['White', 'Black', 'Brown', 'Navy Blue'],
    'Khaki': ['White', 'Navy Blue', 'Brown'],
    'Lime Green': ['White', 'Black', 'Grey'],
    'Peach': ['Blue', 'Black', 'Navy Blue'],
    'Sea Green': ['White', 'Black', 'Navy Blue'],
    'Teal': ['White', 'Cream', 'Navy Blue'],
    'Mauve': ['White', 'Grey', 'Blue'],
    'Copper': ['White', 'Black', 'Navy Blue'],
    'Steel': ['Black', 'White', 'Beige'],
    'Bronze': ['White', 'Black'],
    'Nude': ['White', 'Black', 'Beige'],
    'Metallic': ['White', 'Black', 'Silver'],
    'Taupe': ['White', 'Black', 'Beige'],
    'Fluorescent Green': ['White', 'Black'],
    'Burgundy': ['White', 'Beige', 'Grey'],
    'Mushroom Brown': ['White', 'Grey', 'Black'],
    'Coffee Brown': ['White', 'Beige', 'Cream']
}

In [27]:
categorical_columns = ['masterCategory', 'subCategory', 'articleType', 'gender', 'baseColour', 'season','usage']
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col + '_encoded'] = le.fit_transform(df[col])
    label_encoders[col] = le

In [8]:
X = df[['masterCategory_encoded', 'subCategory_encoded', 'articleType_encoded', 
        'gender_encoded', 'baseColour_encoded', 'season_encoded', 'usage_encoded', 'Price (USD)']]
y = to_categorical(df['subCategory_encoded'])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
def create_dnn_model(num_master_cat, num_sub_cat, num_article_type, num_gender, num_color, num_season, num_usage):
    # Input layers for each feature
    master_cat_input = Input(shape=(1,), name='masterCategory_input')
    sub_cat_input = Input(shape=(1,), name='subCategory_input')
    article_type_input = Input(shape=(1,), name='articleType_input')
    gender_input = Input(shape=(1,), name='gender_input')
    color_input = Input(shape=(1,), name='color_input')
    season_input = Input(shape=(1,), name='season_input')
    usage_input = Input(shape=(1,), name='usage_input')
    price_input = Input(shape=(1,), name='price_input')

    # Embedding layers for categorical features
    master_cat_embed = Embedding(num_master_cat, 10)(master_cat_input)
    sub_cat_embed = Embedding(num_sub_cat, 8)(sub_cat_input)
    article_type_embed = Embedding(num_article_type, 8)(article_type_input)
    gender_embed = Embedding(num_gender, 4)(gender_input)
    color_embed = Embedding(num_color, 4)(color_input)
    season_embed = Embedding(num_season, 4)(season_input)
    usage_embed = Embedding(num_usage, 4)(usage_input)

    # Flatten embeddings
    master_cat_flat = Flatten()(master_cat_embed)
    sub_cat_flat = Flatten()(sub_cat_embed)
    article_type_flat = Flatten()(article_type_embed)
    gender_flat = Flatten()(gender_embed)
    color_flat = Flatten()(color_embed)
    season_flat = Flatten()(season_embed)
    usage_flat = Flatten()(usage_embed)

    # Concatenate all inputs
    concatenated = Concatenate()([
        master_cat_flat, sub_cat_flat, article_type_flat,
        gender_flat, color_flat, season_flat, usage_flat, price_input
    ])

    # Dense layers with dropout and batch normalization
    x = Dense(64, activation='relu')(concatenated)
    x = Dropout(0.3)(x)
    x = BatchNormalization()(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = BatchNormalization()(x)

    # Output layer for predicting complementary subcategory
    output = Dense(num_sub_cat, activation='softmax', name='subCategory_output')(x)

    # Create and compile the model
    model = Model(inputs=[
        master_cat_input, sub_cat_input, article_type_input,
        gender_input, color_input, season_input, usage_input, price_input
    ], outputs=output)

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [11]:
num_master_cat = df['masterCategory_encoded'].nunique()
num_sub_cat = df['subCategory_encoded'].nunique()
num_article_type = df['articleType_encoded'].nunique()
num_gender = df['gender_encoded'].nunique()
num_color = df['baseColour_encoded'].nunique()
num_season = df['season_encoded'].nunique()
num_usage = df['usage_encoded'].nunique()

In [12]:
model = create_dnn_model(num_master_cat, num_sub_cat, num_article_type, num_gender, num_color, num_season, num_usage)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [13]:
X_train_inputs = [np.array(X_train[col]) for col in X_train.columns]
X_test_inputs = [np.array(X_test[col]) for col in X_test.columns]

In [14]:
model.fit(X_train_inputs, y_train, validation_data=(X_test_inputs, y_test), epochs=10, batch_size=32)

Train on 29187 samples, validate on 7297 samples
Epoch 1/10
29187/29187 [==============================] - 4s 132us/sample - loss: 0.6304 - acc: 0.8219 - val_loss: 0.0277 - val_acc: 1.0000
Epoch 2/10
29187/29187 [==============================] - 3s 120us/sample - loss: 0.1753 - acc: 0.9514 - val_loss: 0.0048 - val_acc: 1.0000
Epoch 3/10
29187/29187 [==============================] - 4s 128us/sample - loss: 0.1035 - acc: 0.9704 - val_loss: 0.0010 - val_acc: 1.0000
Epoch 4/10
29187/29187 [==============================] - 3s 111us/sample - loss: 0.0765 - acc: 0.9783 - val_loss: 0.0093 - val_acc: 0.9999
Epoch 5/10
29187/29187 [==============================] - 3s 104us/sample - loss: 0.0694 - acc: 0.9804 - val_loss: 2.0254e-04 - val_acc: 1.0000
Epoch 6/10
29187/29187 [==============================] - 3s 102us/sample - loss: 0.0620 - acc: 0.9833 - val_loss: 2.7035e-04 - val_acc: 1.0000
Epoch 7/10
29187/29187 [==============================] - 3s 103us/sample - loss: 0.0550 - acc: 0.9855 

In [ ]:
loss, accuracy = model.evaluate(X_val, y_val) 
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

7297/7297 [==============================] - 0s 17us/sample - loss: 0.0068 - acc: 0.9982
Validation Loss: 0.006824821795151291
Validation Accuracy: 0.9982184171676636


In [32]:
def recommend_complementary_based_on_map(item_id, df, complementary_map, colour_contrast_map):
    # Get the original item details
    item = df[df['id'] == item_id].iloc[0]
    
    # Get complementary subcategories based on the item's subcategory
    complementary_subcategories = complementary_map.get(item['subCategory'], [])
    
    recommendations = []

    for subCategory in complementary_subcategories:
        # Filter dataset for items matching the complementary subcategory and other features
        complementary_items = df[(df['subCategory'] == subCategory) & 
                                 (df['gender'] == item['gender']) & 
                                 (df['season'] == item['season']) & 
                                 (df['usage'] == item['usage'])]
        
        # Apply color contrast map to filter based on complementary colors
        complementary_colors = colour_contrast_map.get(item['baseColour'], [])
        color_filtered_items = complementary_items[complementary_items['baseColour'].isin(complementary_colors)]
        
        if not color_filtered_items.empty:
            # Recommend the first item for each subcategory based on color contrast filtering
            recommendations.append((subCategory, color_filtered_items[['id']].head(1).to_dict(orient="records")))

    return recommendations





In [31]:
item_id = 8493  
recommendations = recommend_complementary_based_on_map(
    item_id, df, complementary_map, colour_contrast_map
)
print("Recommendations:", recommendations)

Recommendations: [('Topwear', [{'id': 13269}]), ('Shoes', [{'id': 17959}]), ('Flip Flops', [{'id': 19866}]), ('Bags', [{'id': 58174}]), ('Belts', [{'id': 20070}]), ('Watches', [{'id': 19292}]), ('Wallets', [{'id': 20861}])]


In [24]:
recommended_product_ids = [8493,13269,17959, 19866,58174,20070,19292,20861]
recommended_products = df[df['id'].isin(recommended_product_ids)]
print(recommended_products)

       user_id     id gender masterCategory subCategory   articleType  \
0        93810   8493  Women        Apparel  Bottomwear        Skirts   
38       23396  13269  Women        Apparel     Topwear          Tops   
1047     81511  58174  Women    Accessories        Bags      Handbags   
1673     91830  17959  Women       Footwear       Shoes  Casual Shoes   
5408     13826  19292  Women    Accessories     Watches       Watches   
6371     85243  19866  Women       Footwear  Flip Flops    Flip Flops   
7444     34176  20861  Women    Accessories     Wallets       Wallets   
32604    70249  20070  Women    Accessories       Belts         Belts   

      baseColour season    year   usage  ...  \
0          Black   Fall  2011.0  Casual  ...   
38         White   Fall  2011.0  Casual  ...   
1047       White   Fall  2012.0  Casual  ...   
1673       White   Fall  2011.0  Casual  ...   
5408       White   Fall  2012.0  Casual  ...   
6371       White   Fall  2011.0  Casual  ...   
7444  

In [ ]:
item_id = 58174  
recommendations = recommend_complementary_based_on_map(
    item_id, df, complementary_map, colour_contrast_map
)
print("Recommendations:", recommendations)

Recommendations: [('Topwear', [{'id': 22215}]), ('Bottomwear', [{'id': 8493}]), ('Shoes', [{'id': 10271}]), ('Watches', [{'id': 19264}]), ('Jewellery', [{'id': 46133}])]


In [26]:
recommended_product_ids = [58174,22215,8493,10271,19264,46133]
recommended_products = df[df['id'].isin(recommended_product_ids)]
print(recommended_products)

       user_id     id gender masterCategory subCategory          articleType  \
0        93810   8493  Women        Apparel  Bottomwear               Skirts   
110      70142  22215  Women        Apparel     Topwear              Tshirts   
254      66498  10271  Women       Footwear       Shoes                Heels   
615      69829  46133  Women    Accessories   Jewellery  Necklace and Chains   
1047     81511  58174  Women    Accessories        Bags             Handbags   
34276    96662  19264  Women    Accessories     Watches              Watches   

      baseColour season    year   usage  ...  \
0          Black   Fall  2011.0  Casual  ...   
110         Blue   Fall  2011.0  Casual  ...   
254          Red   Fall  2012.0  Casual  ...   
615        Black   Fall  2012.0  Casual  ...   
1047       White   Fall  2012.0  Casual  ...   
34276      Black   Fall  2012.0  Casual  ...   

                                                  review     Month  \
0      Amazing quality! Definite

In [ ]:
# recommended_product_ids = [8493,51610, 21183,19773,14339]
# recommended_products = df[df['id'].isin(recommended_product_ids)]
# print(recommended_products)

       user_id     id gender masterCategory subCategory articleType  \
0        93810   8493  Women        Apparel  Bottomwear      Skirts   
1246     96991  19773  Women        Apparel  Bottomwear   Stockings   
10236    35969  21183  Women       Footwear  Flip Flops  Flip Flops   
12383    77215  14339  Women       Footwear  Flip Flops  Flip Flops   
33357    33568  51610  Women        Apparel  Bottomwear    Trousers   

      baseColour season    year   usage  ...  \
0          Black   Fall  2011.0  Casual  ...   
1246        Grey   Fall  2011.0  Casual  ...   
10236       Blue   Fall  2010.0  Casual  ...   
12383       Grey   Fall  2011.0  Casual  ...   
33357      Brown   Fall  2012.0  Casual  ...   

                                                  review     Month  \
0      Amazing quality! Definitely recommend. Would r...  November   
1246   Comfortable and stylish, worth the price. and ...   October   
10236  Perfect for any occasion, great buy. Would rec...  November   
1238